In [1]:
import pandas as pd
import numpy as np
departments = pd.read_csv('./data/departments.csv', sep=',', encoding="ISO-8859-1")
products = pd.read_csv('./data/products.csv', sep=',', encoding="ISO-8859-1")
mergedOrders = pd.read_csv('./data/mergedOrders.csv', sep=',', encoding="ISO-8859-1")
orders1 = pd.read_csv('./data/orders.csv', sep=',', encoding="ISO-8859-1")

In [62]:
products.head()
#departments.head()
#mergedOrders.head()

,department_id,product_id,product_name
0,19,1,Chocolate Sandwich Cookies
1,13,2,All-Seasons Salt
2,7,3,Robust Golden Unsweetened Oolong Tea
3,1,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,13,5,Green Chile Anytime Sauce


In [3]:
#mergedOrders = mergedOrders.drop(columns = ["order_id"])
dfDaysAverage = mergedOrders.groupby("product_id").mean()
dfDaysAverage.sort_values("days_since_prior_order").dropna()
dfDaysAverage = dfDaysAverage.drop(columns=["Unnamed: 0","order_id","reordered","user_id"])
dfDaysAverage = dfDaysAverage.rename(columns={"days_since_prior_order": "mediaEcompras"})
dfDaysAverage = dfDaysAverage[dfDaysAverage["mediaEcompras"] > 0]
dfDaysAverage['product_id'] = dfDaysAverage.index
dfDaysAverage.head()

,mediaEcompras,product_id
product_id,,
1,12.333333,1
2,20.500000,2
3,19.250000,3
4,12.000000,4
7,10.666667,7


In [4]:
dfQuantity = mergedOrders.groupby("product_id").count()
dfQuantity = dfQuantity.drop(columns=["Unnamed: 0","order_id","reordered","days_since_prior_order"]).reset_index()
dfQuantity = dfQuantity.rename(columns={"user_id": "cantitatMitjaProductes"})
dfQuantity = dfQuantity.sort_values('cantitatMitjaProductes')
dfQuantity['product_id'] = dfQuantity.index
dfQuantity.head()

,product_id,cantitatMitjaProductes
20144,20144,1
28037,28037,1
13241,13241,1
13242,13242,1
13243,13243,1


In [5]:
allMergedOrders = mergedOrders
aux = allMergedOrders.groupby( [ "user_id", "product_id"] ).size().to_frame(name = 'total').reset_index()
allMergedOrders = allMergedOrders.groupby( [ "product_id", "user_id"] ).mean().reset_index()
allMergedOrders = allMergedOrders.drop(columns=["Unnamed: 0","order_id","reordered"]).reset_index()
allMergedOrders = pd.merge(aux,allMergedOrders, on=[ "user_id", "product_id"])
allMergedOrders = allMergedOrders.rename(columns={"days_since_prior_order": "media_usuarioEcompras"})
allMergedOrders = allMergedOrders.drop(columns=["index"])
allMergedOrders.head()

,user_id,product_id,total,media_usuarioEcompras
0,1,196,11,19.000000
1,1,10258,10,19.000000
2,1,10326,1,28.000000
3,1,12427,10,19.555556
4,1,13032,4,19.750000


In [6]:
def puntTemps(df):
    a2 = df['mediaEcompras'] / 2

    if df['media_usuarioEcompras'] <= df['mediaEcompras']: 
        return 5
    elif df['media_usuarioEcompras'] < df['mediaEcompras'] + a2:
        return 3
    else: 
        return 1

def puntQuant(df):
    a2 = df['cantitatMitjaProductes'] / 2
    
    if df['total'] < a2: 
        return 1
    elif df['total'] < df['cantitatMitjaProductes']:
        return 3
    else: 
        return 5

In [7]:
dfDaysAverage.index.name = None
dfProva = allMergedOrders
#dfProva = dfProva.groupby(["user_id",'product_id']).agg({'product_id':['mean', 'median']})
#dfProva['puntuacio'] = dfProva.age * dfProva.bmi
#dfProva = dfProva.groupby( [ "user_id", "product_id"] ).size().to_frame(name = 'total').reset_index()
dfProva =  pd.merge(dfProva, dfQuantity, on="product_id")
dfProva =  pd.merge(dfProva, dfDaysAverage, on="product_id")
dfProva['punCant'] = dfProva.apply(puntQuant,axis=1)
dfProva['punTemps'] = dfProva.apply(puntTemps,axis=1)
dfProva['punTotal'] = dfProva.punTemps + dfProva.punCant
dfProva

,user_id,product_id,total,media_usuarioEcompras,cantitatMitjaProductes,mediaEcompras,punCant,punTemps,punTotal
0,1,196,11,19.000000,9,12.160839,5,1,6
1,15,196,5,8.600000,9,12.160839,3,5,8
2,19,196,3,6.666667,9,12.160839,1,5,6
3,21,196,1,6.000000,9,12.160839,1,5,6
4,31,196,2,7.500000,9,12.160839,1,5,6
...,...,...,...,...,...,...,...,...,...
245292,6038,4078,1,5.000000,39,5.000000,1,5,6
245293,6038,19526,1,15.000000,2,15.000000,3,5,8
245294,6038,20096,2,7.500000,1,7.500000,5,5,10
245295,6043,21628,1,30.000000,4,30.000000,1,5,6


In [8]:
dfTotal = dfQuantity
dfTotal = pd.merge(dfDaysAverage, dfTotal, on = "product_id")

In [9]:
dfTotal = dfTotal.sort_values('cantitatMitjaProductes')
dfTotal

,mediaEcompras,product_id,cantitatMitjaProductes
12369,9.125000,20334,1
11605,13.500000,19025,1
11611,26.000000,19032,1
11612,11.600000,19033,1
4279,20.666667,7046,1
...,...,...,...
18280,15.000000,29910,5065
18054,14.335227,29553,6813
8307,30.000000,13695,7115
7992,23.000000,13191,8182


In [10]:
dfPuntUsuariTotal = dfProva.drop(columns=["total","media_usuarioEcompras","cantitatMitjaProductes", "mediaEcompras","punCant", "punTemps"])
dfPuntUsuariTotal

,user_id,product_id,punTotal
0,1,196,6
1,15,196,8
2,19,196,6
3,21,196,6
4,31,196,6
...,...,...,...
245292,6038,4078,6
245293,6038,19526,8
245294,6038,20096,10
245295,6043,21628,6


In [28]:
from sklearn.model_selection import train_test_split

ratings = dfPuntUsuariTotal
#train, test = train_test_split(ratings, test_size = 0.10)
#ratings = pd.merge(dfPuntUsuariTotal, test)


ratings[ratings['product_id'] == 15596]
ratings

,user_id,product_id,punTotal
0,1,196,6
1,15,196,8
2,19,196,6
3,21,196,6
4,31,196,6
...,...,...,...
245292,6038,4078,6
245293,6038,19526,8
245294,6038,20096,10
245295,6043,21628,6


In [67]:
ratingsaux =ratings.groupby('product_id').count().sort_values('user_id')
ratingsaux = ratingsaux[ratingsaux['user_id'] > 3]
#ratings2 = ratings[ratings['product_id'].isin(list(ratingsaux.index))]
#ratings2
print(len(list(ratingsaux.index)))
usefullProducts = products[products['product_id'].isin(list(ratingsaux.index))] 
usefullProducts.to_csv("usefullProducts.csv")

8787


In [52]:
userRatings = ratings2.pivot_table(index=['user_id'], columns=['product_id'], values='punTotal')

In [53]:
userRatings[196].dropna()

user_id
1       6.0
15      8.0
19      6.0
21      6.0
31      6.0
       ... 
5844    6.0
5890    6.0
5920    6.0
5997    6.0
6041    6.0
Name: 196, Length: 244, dtype: float64

In [54]:
userRatings

product_id,1,4,10,11,23,25,26,28,29,34,...,31060,31066,31067,31073,31074,31075,31076,31081,31086,31087
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
corrMatrix = userRatings.corr(method='pearson', min_periods=1)
corrMatrix

product_id,1,4,10,11,23,25,26,28,29,34,...,31060,31066,31067,31073,31074,31075,31076,31081,31086,31087
product_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
31076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
31081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [56]:
corrMatrix.to_csv("MatrixBigger3.txt")

In [78]:
#myRatings = userRatings.loc[0].dropna()
#myRatings
myRatings = {196: 10}
myRatings
corrMatrix[196].dropna()

product_id
1        0.600481
54       0.866025
116      0.673124
130      0.279852
148     -1.000000
           ...   
30777   -0.688247
30827    0.562970
30960   -0.353553
31060    1.000000
31066    0.566564
Name: 196, Length: 855, dtype: float64

In [74]:
simCandidates = pd.Series()
print(simCandidates)
for key in myRatings:
    print ("Añadiendo productos similares a " + str(key) + "...")
    sims = corrMatrix[key].dropna()
    sims = sims.map(lambda x: x * myRatings[key])
    simCandidates = simCandidates.append(sims)
    

print ("ordenando...")
simCandidates.sort_values(inplace = True, ascending = False)
print (simCandidates.head(1000))

Series([], dtype: float64)
Añadiendo productos similares a 196...
ordenando...
4938     10.0
19604    10.0
21784    10.0
21883    10.0
22000    10.0
         ... 
6576    -10.0
5748    -10.0
24660   -10.0
28031   -10.0
15097   -10.0
Length: 855, dtype: float64


<ipython-input-74-a526dbb9b3f6>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  simCandidates = pd.Series()


In [105]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()
dfAuxCount = mergedOrders.groupby("product_id").count()
dfAuxCount["product_id"] = dfAuxCount.index
dfAuxCount = dfAuxCount[dfAuxCount['product_id'].isin(list(simCandidates.index))]
dfAuxCount
simCandidates = simCandidates.to_frame('puntuacio')
simCandidates["product_id"] = simCandidates.index


AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [126]:
dfAuxCount = dfAuxCount.rename_axis(None)
dfFinal = pd.merge(simCandidates, dfAuxCount, on="product_id")
dfFinal= dfFinal.astype({'puntuacio': 'int32'})
dfFinal

,level_0,index,puntuacio,product_id,cantitat
0,0,1,6,1,62
1,1,54,8,54,51
2,2,116,6,116,129
3,3,130,2,130,105
4,4,148,-10,148,168
...,...,...,...,...,...
850,850,30777,-6,30777,42
851,851,30827,5,30827,447
852,852,30960,-3,30960,111
853,853,31060,10,31060,24


In [127]:
#simCandidates= simCandidates.reset_index()
#dfAuxCount = dfAuxCount.reset_index()

dfFinal.sort_values(by=['puntuacio','cantitat'], ascending=[False,False])

,level_0,index,puntuacio,product_id,cantitat
833,833,30391,10,30391,2553
5,5,196,10,196,1070
541,541,19678,10,19678,893
827,827,30233,10,30233,814
134,134,4957,10,4957,681
...,...,...,...,...,...
520,520,18766,-10,18766,7
660,660,23874,-10,23874,7
80,80,3050,-10,3050,6
617,617,22318,-10,22318,5


In [121]:
dfAuxCount.drop(columns=["Unnamed: 0","order_id","reordered","days_since_prior_order"], inplace=True)
dfAuxCount.rename(columns={'user_id': 'cantitat'}, inplace=True)

KeyError: "['Unnamed: 0' 'order_id' 'reordered' 'days_since_prior_order'] not found in axis"

In [99]:
dfAuxCount.sort_values


,user_id,product_id
product_id,,
1,62,1
54,51,54
116,129,116
130,105,130
148,168,148
...,...,...
30777,42,30777
30827,447,30827
30960,111,30960


In [101]:
simCandidates.sort_values(inplace = True, ascending = False)
simCandidates.head(10)
#items = simCandidates.items()
#for item in items:
#   print(item[0])

30720    10.0
10814    10.0
20580    10.0
19706    10.0
19678    10.0
19003    10.0
18441    10.0
17828    10.0
16349    10.0
15683    10.0
dtype: float64

In [102]:
print(myRatings.keys())

dict_keys([24852])


In [95]:
filteredSims = simCandidates.drop(myRatings.keys())
filteredSims.head(10)
#print(simCandidates.to_string())

30720    1.0
10814    1.0
20580    1.0
19706    1.0
19678    1.0
19003    1.0
18441    1.0
17828    1.0
16349    1.0
15683    1.0
dtype: float64